

---

# **Chapter 8: Algorithms and Responsibilities**

## **Opening Context**
Algorithms are the soul of software, yet they are also a primary source of rigidity. When business rules become hard-coded in conditional statements, when variations of a process require copy-paste inheritance, or when direct method calls create unbreakable coupling between sender and receiver, systems become fragile. Behavioral patterns address these challenges by shifting the focus from object composition to communication and responsibility distribution. This chapter examines three foundational patterns: Strategy for interchangeable algorithms, Template Method for algorithmic skeletons, and Command for encapsulating requests as first-class objects. Together, they form the toolkit for making behavior as flexible as structure.

---

## **8.1 Strategy Pattern: Swappable Algorithms**

### **Intent**
*Define a family of algorithms, encapsulate each one, and make them interchangeable. Strategy lets the algorithm vary independently from clients that use it.*

### **The Problem**
Consider a payment processing system that initially only supports credit cards. As the business grows, it must support PayPal, cryptocurrency, and Buy Now Pay Later (BNPL). The naive approach uses conditional statements:

```typescript
// ❌ CONDITIONAL HELL: Violates OCP, SRP, and becomes unmaintainable
class PaymentProcessor {
    process(amount: number, method: 'credit' | 'paypal' | 'crypto'): void {
        if (method === 'credit') {
            console.log(`Processing $${amount} via Credit Card`);
            // 20 lines of credit-specific logic
            this.validateCard();
            this.chargeCard(amount);
        } else if (method === 'paypal') {
            console.log(`Processing $${amount} via PayPal`);
            // 15 lines of PayPal API integration
            this.redirectToPayPal();
            this.capturePayPalPayment(amount);
        } else if (method === 'crypto') {
            console.log(`Processing $${amount} via Crypto`);
            // 25 lines of blockchain interaction
            this.validateWallet();
            this.transferCrypto(amount);
        }
        // Adding new method requires modifying this class (OCP violation)
    }
    
    // Payment-specific methods pollute the class
    private validateCard(): void {}
    private chargeCard(amount: number): void {}
    private redirectToPayPal(): void {}
    private capturePayPalPayment(amount: number): void {}
    private validateWallet(): void {}
    private transferCrypto(amount: number): void {}
}
```

**Problems**:
- **OCP Violation**: Adding a payment method modifies the existing class.
- **SRP Violation**: Class handles payment orchestration AND specific implementations.
- **Testing Difficulty**: Cannot test PayPal logic without Credit Card dependencies loaded.
- **Code Duplication**: Common steps (logging, validation) repeated or scattered.

### **The Solution: Algorithm Encapsulation**

The Strategy pattern extracts algorithms into separate, interchangeable classes that share a common interface.

```typescript
/**
 * STRATEGY INTERFACE
 * Declares the contract common to all supported algorithms.
 * Context uses this interface to call the algorithm defined by Concrete Strategies.
 */
interface PaymentStrategy {
    pay(amount: number): PaymentResult;
    validate(): boolean;
    getName(): string;
}

interface PaymentResult {
    success: boolean;
    transactionId: string;
    message: string;
}

/**
 * CONCRETE STRATEGIES
 * Implement the algorithm using the Strategy interface.
 */
class CreditCardStrategy implements PaymentStrategy {
    private cardNumber: string;
    private cvv: string;
    private expiryDate: string;
    
    constructor(cardNumber: string, cvv: string, expiry: string) {
        this.cardNumber = cardNumber;
        this.cvv = cvv;
        this.expiryDate = expiry;
    }
    
    validate(): boolean {
        // Luhn algorithm validation
        return this.cardNumber.length === 16 && this.cvv.length === 3;
    }
    
    pay(amount: number): PaymentResult {
        console.log(`[CreditCard] Charging $${amount} to card ending in ${this.cardNumber.slice(-4)}`);
        // Actual credit card processing logic here
        return {
            success: true,
            transactionId: `CC-${Date.now()}`,
            message: "Payment processed successfully"
        };
    }
    
    getName(): string {
        return "Credit Card";
    }
}

class PayPalStrategy implements PaymentStrategy {
    private email: string;
    private password: string;
    
    constructor(email: string, password: string) {
        this.email = email;
        this.password = password;
    }
    
    validate(): boolean {
        return this.email.includes('@');
    }
    
    pay(amount: number): PaymentResult {
        console.log(`[PayPal] Processing $${amount} for account ${this.email}`);
        // PayPal API integration
        return {
            success: true,
            transactionId: `PP-${Date.now()}`,
            message: "PayPal payment confirmed"
        };
    }
    
    getName(): string {
        return "PayPal";
    }
}

class CryptoStrategy implements PaymentStrategy {
    private walletAddress: string;
    private cryptoType: 'BTC' | 'ETH';
    
    constructor(wallet: string, type: 'BTC' | 'ETH') {
        this.walletAddress = wallet;
        this.cryptoType = type;
    }
    
    validate(): boolean {
        return this.walletAddress.startsWith('0x') || this.walletAddress.startsWith('1');
    }
    
    pay(amount: number): PaymentResult {
        console.log(`[Crypto] Transferring $${amount} equivalent in ${this.cryptoType}`);
        // Blockchain interaction
        return {
            success: true,
            transactionId: `CRYPTO-${Date.now()}`,
            message: `Blockchain transaction confirmed: ${this.cryptoType}`
        };
    }
    
    getName(): string {
        return `Cryptocurrency (${this.cryptoType})`;
    }
}

/**
 * CONTEXT
 * Maintains a reference to a Strategy object.
 * Delegates the algorithm execution to the strategy instance.
 */
class ShoppingCart {
    private items: Array<{ name: string; price: number }> = [];
    private paymentStrategy: PaymentStrategy;
    
    // Strategy injected via constructor (DIP compliance)
    constructor(strategy: PaymentStrategy) {
        this.paymentStrategy = strategy;
    }
    
    // Allows runtime strategy switching
    setPaymentStrategy(strategy: PaymentStrategy): void {
        this.paymentStrategy = strategy;
    }
    
    addItem(name: string, price: number): void {
        this.items.push({ name, price });
    }
    
    calculateTotal(): number {
        return this.items.reduce((sum, item) => sum + item.price, 0);
    }
    
    checkout(): PaymentResult {
        const total = this.calculateTotal();
        
        console.log(`\nChecking out ${this.items.length} items. Total: $${total}`);
        console.log(`Using payment method: ${this.paymentStrategy.getName()}`);
        
        // Delegation to strategy
        if (!this.paymentStrategy.validate()) {
            return {
                success: false,
                transactionId: '',
                message: "Payment validation failed"
            };
        }
        
        return this.paymentStrategy.pay(total);
    }
}

// Usage: Runtime algorithm selection
const cart = new ShoppingCart(new CreditCardStrategy("1234567890123456", "123", "12/25"));

cart.addItem("Laptop", 1200);
cart.addItem("Mouse", 25);

// Checkout with credit card
let result = cart.checkout();

// Switch strategy at runtime based on user choice
cart.setPaymentStrategy(new PayPalStrategy("user@example.com", "secret"));
cart.addItem("Keyboard", 75);
result = cart.checkout();

// Or crypto
cart.setPaymentStrategy(new CryptoStrategy("0x123abc", "ETH"));
result = cart.checkout();
```

**Code Explanation**:

- **Strategy Interface** (`PaymentStrategy`): Declares the contract (`pay`, `validate`). This abstraction allows the Context to work with any algorithm variant without knowing concrete details (DIP compliance).
- **Concrete Strategies** (`CreditCardStrategy`, etc.): Encapsulate specific algorithms. Each is self-contained, testable independently, and adheres to SRP.
- **Context** (`ShoppingCart`): Maintains a reference to the current strategy and delegates algorithm execution. The context is decoupled from specific payment implementations—it knows only that it has a `PaymentStrategy`.

**Strategy Selection Strategies**:
1.  **Constructor Injection**: Client chooses strategy at creation time.
2.  **Setter Injection**: Runtime switching (shown above).
3.  **Factory Integration**: Strategy determined by configuration or user input via Factory Method (Chapter 4).

### **Connection to SOLID**
- **OCP**: New payment methods require only new strategy classes, no changes to `ShoppingCart`.
- **DIP**: `ShoppingCart` depends on `PaymentStrategy` (abstraction), not concrete implementations.
- **SRP**: Each strategy handles one payment type; `ShoppingCart` handles cart logic, not payment processing.

### **Strategy vs. State Pattern**
Both patterns delegate behavior to contained objects with similar structures. The difference is **intent**:
- **Strategy**: Client chooses algorithm based on preference/optimization (e.g., sorting strategy).
- **State**: Object changes behavior based on internal state transitions (e.g., TCP connection states: Established, Closed, Listening).

---

## **8.2 Template Method Pattern: Skeleton of an Algorithm**

### **Intent**
*Define the skeleton of an algorithm in an operation, deferring some steps to subclasses. Template Method lets subclasses redefine certain steps of an algorithm without changing the algorithm's structure.*

### **The Problem**
Multiple algorithms share the same high-level steps but differ in specific implementations. Without Template Method, this results in duplication of the algorithm structure or complex conditionals:

```typescript
// ❌ CODE DUPLICATION: Same structure, different steps
class CSVDataMiner {
    mine(path: string): void {
        const file = this.openFile(path); // Common
        const rawData = this.extractData(file); // Specific
        const data = this.parseData(rawData); // Specific
        const analysis = this.analyze(data); // Common
        this.sendReport(analysis); // Common
        this.closeFile(file); // Common
    }
    // ... implementations
}

class PDFDataMiner {
    mine(path: string): void {
        const file = this.openFile(path); // Same
        const rawData = this.extractPDFData(file); // Different
        const data = this.parsePDFData(rawData); // Different
        const analysis = this.analyze(data); // Same
        this.sendReport(analysis); // Same
        this.closeFile(file); // Same
    }
}
```

### **The Solution: Algorithmic Skeleton**

The Template Method defines the algorithm steps in a base class, marking invariant steps as `final` (or non-overridable) and variant steps as `abstract` or `protected hook methods`.

```typescript
/**
 * ABSTRACT CLASS
 * Defines the template method (mine) with the algorithm skeleton.
 * Declares abstract operations for steps that vary.
 * Implements common steps or provides default behavior.
 */
abstract class DataMiner {
    /**
     * TEMPLATE METHOD
     * Defines the algorithm structure. Declared final to prevent
     * subclasses from changing the sequence.
     */
    mine(path: string): void {
        // Step 1: Common
        const file = this.openFile(path);
        
        try {
            // Step 2: Customizable (Hook)
            const rawData = this.extractData(file);
            
            // Step 3: Customizable (Hook)
            const data = this.parse(rawData);
            
            // Step 4: Common with optional hook
            const analysis = this.analyze(data);
            
            // Step 5: Hook with default implementation
            if (this.shouldSendReport()) {
                this.sendReport(analysis);
            }
        } finally {
            // Step 6: Common (cleanup guaranteed)
            this.closeFile(file);
        }
    }
    
    // Common steps (final/private)
    private openFile(path: string): File {
        console.log(`Opening file: ${path}`);
        return new File(path);
    }
    
    private analyze(data: any[]): any {
        console.log("Performing common analysis...");
        return { count: data.length, summary: "processed" };
    }
    
    private closeFile(file: File): void {
        console.log("Closing file");
    }
    
    // Abstract methods - must be implemented by subclasses (variant steps)
    protected abstract extractData(file: File): string;
    protected abstract parse(rawData: string): any[];
    
    // Hook method - default implementation, can be overridden
    protected shouldSendReport(): boolean {
        return true;
    }
    
    // Another hook with empty default (optional step)
    protected sendReport(analysis: any): void {
        console.log("Sending report:", analysis);
    }
}

/**
 * CONCRETE CLASSES
 * Implement primitive operations to carry out subclass-specific steps.
 */
class CSVDataMiner extends DataMiner {
    protected extractData(file: File): string {
        console.log("Reading CSV lines...");
        return "name,age\nJohn,30\nJane,25";
    }
    
    protected parse(rawData: string): any[] {
        console.log("Parsing CSV into objects...");
        return rawData.split('\n').slice(1).map(line => {
            const [name, age] = line.split(',');
            return { name, age: parseInt(age) };
        });
    }
    
    // Override hook to customize behavior
    protected shouldSendReport(): boolean {
        return false; // CSV reports generated locally only
    }
}

class PDFDataMiner extends DataMiner {
    protected extractData(file: File): string {
        console.log("Extracting text from PDF...");
        return "PDF content parsed";
    }
    
    protected parse(rawData: string): any[] {
        console.log("Parsing PDF structure...");
        return [{ type: 'pdf', content: rawData }];
    }
    
    protected sendReport(analysis: any): void {
        console.log("Sending PDF-specific formatted report:", analysis);
    }
}

// Support class
class File {
    constructor(public path: string) {}
}

// Usage
console.log("=== CSV Mining ===");
const csvMiner = new CSVDataMiner();
csvMiner.mine("data.csv");

console.log("\n=== PDF Mining ===");
const pdfMiner = new PDFDataMiner();
pdfMiner.mine("document.pdf");
```

**Code Explanation**:

- **Template Method** (`mine`): Defines the fixed algorithm sequence (open → extract → parse → analyze → send → close). It calls primitive operations, concrete operations, and hooks.
- **Primitive Operations** (`extractData`, `parse`): Abstract methods that subclasses must implement to provide specific behavior.
- **Concrete Operations** (`openFile`, `analyze`, `closeFile`): Common steps implemented in base class, shared by all subclasses.
- **Hook Methods** (`shouldSendReport`, `sendReport`): Optional steps with default implementations. Subclasses may override to provide custom behavior or disable steps (e.g., `shouldSendReport` returning false).

**Hollywood Principle**: "Don't call us, we'll call you." The parent class (superclass) controls the overall process and calls child class (subclass) methods when needed, rather than the child calling parent methods.

### **Template Method vs. Strategy**
| Template Method | Strategy |
|----------------|----------|
| Inheritance-based (is-a relationship) | Composition-based (has-a relationship) |
| Varies parts of algorithm via subclassing | Varies entire algorithm via delegation |
| Control inverted (parent controls flow) | Client controls which strategy to use |
| Code reuse via inheritance | Code reuse via composition |

**Guideline**: Use Template Method when subclasses share the same algorithm structure; use Strategy when you need to swap entire algorithms at runtime or when classes don't share a common structure.

---

## **8.3 Command Pattern: Encapsulating Requests as Objects**

### **Intent**
*Encapsulate a request as an object, thereby letting you parameterize clients with different requests, queue or log requests, and support undoable operations.*

### **The Problem**
Direct method calls create tight coupling between the invoker (UI button) and the receiver (business logic):

```typescript
// ❌ TIGHT COUPLING: Button knows about Light
class Button {
    private light: Light;
    
    constructor(light: Light) {
        this.light = light;
    }
    
    click(): void {
        this.light.turnOn(); // Hard-coded action
    }
}

// Problems:
// 1. Cannot assign different actions to buttons at runtime
// 2. Cannot queue actions for later execution
// 3. Cannot implement undo
// 4. Cannot log actions for audit trails
```

### **The Solution: Command as Object**

The Command pattern transforms a request into a stand-alone object containing all information about the request. This decouples sender from receiver.

```typescript
/**
 * COMMAND INTERFACE
 * Declares the interface for executing an operation.
 */
interface Command {
    execute(): void;
    undo(): void; // For undoable operations
    getName(): string;
}

/**
 * RECEIVER
 * Knows how to perform the operations associated with carrying out a request.
 * Any class may serve as a Receiver.
 */
class Light {
    private location: string;
    private isOn: boolean = false;
    
    constructor(location: string) {
        this.location = location;
    }
    
    turnOn(): void {
        this.isOn = true;
        console.log(`${this.location} light is ON`);
    }
    
    turnOff(): void {
        this.isOn = false;
        console.log(`${this.location} light is OFF`);
    }
    
    getState(): boolean {
        return this.isOn;
    }
}

class Fan {
    private speed: number = 0;
    private previousSpeed: number = 0;
    
    high(): void {
        this.previousSpeed = this.speed;
        this.speed = 3;
        console.log("Fan is on HIGH");
    }
    
    low(): void {
        this.previousSpeed = this.speed;
        this.speed = 1;
        console.log("Fan is on LOW");
    }
    
    off(): void {
        this.previousSpeed = this.speed;
        this.speed = 0;
        console.log("Fan is OFF");
    }
    
    getSpeed(): number {
        return this.speed;
    }
    
    getPreviousSpeed(): number {
        return this.previousSpeed;
    }
}

/**
 * CONCRETE COMMANDS
 * Binds a Receiver object to an action.
 * Implements execute() by invoking corresponding receiver operations.
 */
class LightOnCommand implements Command {
    private light: Light;
    
    constructor(light: Light) {
        this.light = light;
    }
    
    execute(): void {
        this.light.turnOn();
    }
    
    undo(): void {
        this.light.turnOff();
    }
    
    getName(): string {
        return "Light On";
    }
}

class LightOffCommand implements Command {
    private light: Light;
    
    constructor(light: Light) {
        this.light = light;
    }
    
    execute(): void {
        this.light.turnOff();
    }
    
    undo(): void {
        this.light.turnOn();
    }
    
    getName(): string {
        return "Light Off";
    }
}

class FanHighCommand implements Command {
    private fan: Fan;
    private previousSpeed: number = 0;
    
    constructor(fan: Fan) {
        this.fan = fan;
    }
    
    execute(): void {
        this.previousSpeed = this.fan.getSpeed();
        this.fan.high();
    }
    
    undo(): void {
        // Restore previous speed
        if (this.previousSpeed === 0) this.fan.off();
        else if (this.previousSpeed === 1) this.fan.low();
        else this.fan.high();
    }
    
    getName(): string {
        return "Fan High";
    }
}

/**
 * No Command (Null Object Pattern)
 * Used when no command is assigned, avoids null checks.
 */
class NoCommand implements Command {
    execute(): void { /* do nothing */ }
    undo(): void { /* do nothing */ }
    getName(): string { return "None"; }
}

/**
 * INVOKER
 * Asks the command to carry out the request.
 * Decoupled from receivers; works with any command.
 */
class RemoteControl {
    private onCommands: Command[] = [];
    private offCommands: Command[] = [];
    private undoCommand: Command;
    
    constructor(slots: number = 7) {
        const noCommand = new NoCommand();
        
        // Initialize with no-ops
        for (let i = 0; i < slots; i++) {
            this.onCommands[i] = noCommand;
            this.offCommands[i] = noCommand;
        }
        this.undoCommand = noCommand;
    }
    
    // Configure buttons at runtime
    setCommand(slot: number, onCommand: Command, offCommand: Command): void {
        this.onCommands[slot] = onCommand;
        this.offCommands[slot] = offCommand;
    }
    
    // Actions
    pressOnButton(slot: number): void {
        this.onCommands[slot].execute();
        this.undoCommand = this.onCommands[slot]; // Store for undo
    }
    
    pressOffButton(slot: number): void {
        this.offCommands[slot].execute();
        this.undoCommand = this.offCommands[slot];
    }
    
    pressUndo(): void {
        console.log("\n[UNDO] Reversing last command...");
        this.undoCommand.undo();
    }
    
    displayStatus(): void {
        console.log("\n----- Remote Control Status -----");
        for (let i = 0; i < this.onCommands.length; i++) {
            console.log(`[Slot ${i}] ON: ${this.onCommands[i].getName()}, OFF: ${this.offCommands[i].getName()}`);
        }
    }
}

// Usage
const remote = new RemoteControl();
const livingRoomLight = new Light("Living Room");
const kitchenLight = new Light("Kitchen");
const ceilingFan = new Fan();

// Create commands
const livingRoomLightOn = new LightOnCommand(livingRoomLight);
const livingRoomLightOff = new LightOffCommand(livingRoomLight);
const kitchenLightOn = new LightOnCommand(kitchenLight);
const kitchenLightOff = new LightOffCommand(kitchenLight);
const fanHigh = new FanHighCommand(ceilingFan);

// Configure remote (runtime assignment)
remote.setCommand(0, livingRoomLightOn, livingRoomLightOff);
remote.setCommand(1, kitchenLightOn, kitchenLightOff);
remote.setCommand(2, fanHigh, new NoCommand());

remote.displayStatus();

// Execute commands
remote.pressOnButton(0);  // Living room on
remote.pressOnButton(2);  // Fan high
remote.pressUndo();       // Fan back to previous state
remote.pressOffButton(0); // Living room off
```

**Code Explanation**:

- **Command** (`Command`): Interface declaring `execute()` and `undo()`. The command object encapsulates the binding between a receiver (`Light`, `Fan`) and an action (`turnOn()`, `high()`).
- **Receiver** (`Light`, `Fan`): The actual objects that perform the work. Commands delegate to receivers.
- **Concrete Command** (`LightOnCommand`, etc.): Binds specific receiver to specific action. Stores state necessary for undo (previous fan speed).
- **Invoker** (`RemoteControl`): Holds commands and invokes them. Knows nothing about lights or fans—only that it has commands that can be executed.

### **Advanced Applications**

**1. Command Queue (Job Scheduler)**:
```typescript
class CommandQueue {
    private queue: Command[] = [];
    
    addCommand(cmd: Command): void {
        this.queue.push(cmd);
    }
    
    processQueue(): void {
        while (this.queue.length > 0) {
            const cmd = this.queue.shift()!;
            cmd.execute(); // Execute asynchronously or sequentially
        }
    }
}
```

**2. Macro Commands (Composite Pattern + Command)**:
```typescript
class MacroCommand implements Command {
    private commands: Command[] = [];
    
    add(cmd: Command): void {
        this.commands.push(cmd);
    }
    
    execute(): void {
        // Execute all commands in sequence
        this.commands.forEach(cmd => cmd.execute());
    }
    
    undo(): void {
        // Undo in reverse order
        [...this.commands].reverse().forEach(cmd => cmd.undo());
    }
    
    getName(): string {
        return `Macro (${this.commands.length} commands)`;
    }
}

// Usage: "Party Mode" - dim lights, turn on music, lock doors
const partyMode = new MacroCommand();
partyMode.add(livingRoomLightOff);
partyMode.add(kitchenLightOff);
// ... add more commands
remote.setCommand(3, partyMode, new NoCommand());
```

**3. Transactional Systems**:
Commands can be logged to disk before execution. If system crashes, replay commands to restore state (Event Sourcing, Chapter 17).

---

## **Chapter Summary**
In this chapter, we established three fundamental Behavioral patterns:

1.  **Strategy**: Encapsulates interchangeable algorithms. Enables runtime selection of behavior and eliminates conditional complexity. Satisfies OCP by allowing new algorithms without modifying contexts. **Status: Essential for algorithmic flexibility and plugin architectures.**

2.  **Template Method**: Defines algorithm skeleton in base class, deferring variant steps to subclasses. Promotes code reuse through inheritance while enforcing algorithm structure. **Status: Ideal for frameworks and workflows with fixed sequences but variable implementations.**

3.  **Command**: Encapsulates requests as objects, decoupling sender from receiver. Enables queuing, logging, undo, and macro operations. Transforms method calls into data that can be manipulated. **Status: Critical for undo systems, job queues, and transactional architectures.**

**Key Distinctions**:
- **Strategy** varies the entire algorithm via composition.
- **Template Method** varies parts of the algorithm via inheritance.
- **Command** turns actions into objects, enabling manipulation of requests (queue, log, undo).

---

## **Next Chapter Preview**
**Chapter 9: Communication and Coordination (Observer, Mediator, Chain of Responsibility, Iterator)**

Continuing with Behavioral patterns, Chapter 9 examines how objects communicate without tight coupling. We explore the **Observer** pattern for publish-subscribe notification systems (foundational to React, Redux, and event-driven architectures), the **Mediator** pattern for centralizing complex object interactions (reducing spaghetti dependencies), the **Chain of Responsibility** for passing requests through processing chains, and the **Iterator** pattern for traversing collections without exposing internal structure. These patterns address the critical challenge of managing interactions between objects that must remain loosely coupled.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='7. dynamic_extensions_and_aggregation.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='9. communication_and_coordination.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
